In [16]:
import sys
import os
import os.path as osp
import sqlite3
import json
import re

from shutil import copyfile

import numpy as np
from PIL import Image

# Number of facial landmarks provided by AFLW dataset
N_LANDMARK = 21

In [17]:
def get_img_size(image_filename):
    im = Image.open(image_filename)
    return im.size[0], im.size[1]

In [18]:
def exec_sqlite_query(cursor, select_str, from_str=None, where_str=None):
    query_str = 'SELECT {}'.format(select_str)
    query_str += ' FROM {}'.format(from_str)
    if where_str:
        query_str += ' WHERE {}'.format(where_str)
    return [row for row in cursor.execute(query_str)]

In [19]:
def progress_updt(msg, total, progress):
    bar_length, status = 20, ""
    progress = float(progress) / float(total)
    if progress >= 1.:
        progress, status = 1, "\r\n"
    block = int(round(bar_length * progress))
    text = "\r{}[{}] {:.0f}% {}".format(msg, "#" * block + "-" * (bar_length - block), round(progress * 100, 0), status)
    sys.stdout.write(text)
    sys.stdout.flush()

In [69]:
rgb_list_21 = [[255, 0, 0],
               [178, 34, 34],
               [255, 20, 147],
               [199, 21, 133],
               [255, 69, 0],
               [255, 255, 0],
               [255, 0, 255],
               [128, 0, 128],
               [75, 0, 130],
               [218, 165, 32],
               [173, 255, 47],
               [0, 255, 0],
               [152, 251, 152],
               [0, 128, 0],
               [154, 205, 50],
               [0, 255, 255],
               [127, 255, 212],
               [72, 209, 204],
               [70, 130, 180],
               [0, 191, 255],
               [30, 144, 255],
               [30, 145, 255]] # +1 face
len(rgb_list_21)

22

In [38]:
size_box = 6

def main():
    # Set up a parser for command line arguments
    isVerbose = 1
    dataset_root = "/Users/nikitakorobkov/Downloads/aflw/data"
    jsonFile = "panoptic_aflwcoco8.json"
    dirForNewJPGs = "allImagesJPG3/"
    dirForNewPNGs = "allImagesPNG3/"

    if isVerbose:
        print("#. Transform AFLW annotations into COCO json format...")

    # Open the original AFLW annotation (sqlite database)
    if isVerbose:
        print("  \\__Open the AFLW SQLight database...", end="")
        sys.stdout.flush()

    conn = sqlite3.connect(osp.join(dataset_root, 'aflw.sqlite'))
    cursor = conn.cursor()
    if isVerbose:
        print("Done!")

    # Build sqlite queries
    select_str = "faces.face_id, " \
                 "imgs.filepath, " \
                 "rect.x, rect.y, " \
                 "rect.w, " \
                 "rect.h, " \
                 "pose.roll, " \
                 "pose.pitch, " \
                 "pose.yaw, " \
                 "metadata.sex"
    from_str = "faces, " \
               "faceimages " \
               "imgs, " \
               "facerect rect, " \
               "facepose pose, " \
               "facemetadata metadata"
    where_str = "faces.file_id = imgs.file_id and " \
                "faces.face_id = rect.face_id and " \
                "faces.face_id = pose.face_id and " \
                "faces.face_id = metadata.face_id"
    query_res = exec_sqlite_query(cursor, select_str, from_str, where_str)

    # Count total number of images in AFLW dataset
    if isVerbose:
        print("  \\__Count total number of images in AFLW database: ", end="")
        sys.stdout.flush()
    total_num_images = 0
    for _ in query_res:
        total_num_images += 1
    if isVerbose:
        print(total_num_images)

    # Output file for appending the file paths of not found images
    not_found_images_file = 'not_found_images_aflw.txt'
    try:
        os.remove(not_found_images_file)
    except OSError:
        pass

    # Temporary dataset variables
    aflw_dataset_dict = dict()

    # Register to dataset_dict
    img_cnt = 0
    for face_id, path, rectx, recty, rectw, recth, roll, pitch, yaw, gender in query_res:

        img_cnt += 1

        # Get current image path
        img_path = osp.join(dataset_root, 'flickr', path)

        # Process current image
        if osp.isfile(img_path):
            img_w, img_h = get_img_size(img_path)

            keypoints = N_LANDMARK * 3 * [0]
            pose = [roll, pitch, yaw]
            gender = 0 if gender == 'm' else 1

            # Register
            aflw_dataset_dict[face_id] = {
                'face_id': face_id,
                'img_path': osp.join('flickr', path),
                'width': img_w,
                'height': img_h,
                'bbox': (rectx, recty, rectw, recth),
                'keypoints': keypoints,
                'pose': pose,
                'gender': gender}

        # If current image file does not exist append the not found images filepaths to `not_found_images_file` and
        # continue with the next image file.
        else:
            with open(not_found_images_file, "a") as out:
                out.write("%s\n" % img_path)
            continue

        # Show progress bar
        if isVerbose:
            progress_updt("  \\__Populate AFLW dataset dictionary...", total_num_images, img_cnt)

    if isVerbose:
        print("  \\__Update AFLW dataset dictionary with keypoints...", end="")
        sys.stdout.flush()

    # Landmark property
    # (Visibility is expressed by lack of the coordinate's row.)
    select_str = "faces.face_id, coords.feature_id, coords.x, coords.y"
    from_str = "faces, featurecoords coords"
    where_str = "faces.face_id = coords.face_id"
    query_res = exec_sqlite_query(cursor, select_str, from_str, where_str)

    # Register to dataset_dict
    invalid_face_ids = list()
    for face_id, feature_id, x, y in query_res:
        assert (1 <= feature_id <= N_LANDMARK)
        if face_id in aflw_dataset_dict:
            idx = feature_id - 1
            aflw_dataset_dict[face_id]['keypoints'][3 * idx] = x
            aflw_dataset_dict[face_id]['keypoints'][3 * idx + 1] = y
            aflw_dataset_dict[face_id]['keypoints'][3 * idx + 2] = 1

        elif face_id not in invalid_face_ids:
            invalid_face_ids.append(face_id)

    if isVerbose:
        print("Done!")

    # Close database
    if isVerbose:
        print("  \\__Close the AFLW SQLight database...", end="")
        sys.stdout.flush()
    cursor.close()
    if isVerbose:
        print("Done!")

    # Close database
    if isVerbose:
        print("  \\__Convert to COCO format...", end="")
        sys.stdout.flush()

    images_list = []
    annotations_list = []
    iterI = 0
    for face_id, face_ann in aflw_dataset_dict.items():
        if iterI > 9998:
            break
        iterI = iterI + 1
        
        img_dir_num = int(face_ann['img_path'].split("/")[1])
        img_file_num = int(re.findall(r'\d+', face_ann['img_path'].split("/")[-1].split(".")[0])[0])
        image_id = int("%d%05d" % (img_dir_num, img_file_num))

        #save jpg
        new_file_name = face_ann['img_path'].split("/")[2]
        if(new_file_name.split(".")[1] == "png"):
            new_file_name = new_file_name.split(".")[0] + ".jpg"
            
        new_path_file = "allImagesJPG3/" + new_file_name
        copyfile(face_ann['img_path'], new_path_file)
        
        images_list.append({'id': image_id,
                            'file_name': new_file_name,
                            'height': face_ann['height'],
                            'width': face_ann['width'],
                            'date_captured': '',
                            'flickr_url': '',
                            'license': 1,
                            'dataset': 'aflw'})

        list_keypoints = []
        one_keypoint = []
        for value in face_ann['keypoints']:
            one_keypoint.append(value)
            if len(one_keypoint)==3:
                list_keypoints.append(one_keypoint)
                one_keypoint= []

        #create Array2PNG
        w, h = face_ann['width'], face_ann['height']
        array2png = np.zeros((h, w, 3), dtype=np.uint8)
        
        segmentations_list = []
        for i, keypoint in enumerate(list_keypoints):
            v=keypoint[2]
            if v == 1: #видно keypoint
                x=int(keypoint[0])
                y=int(keypoint[1])
                segmentation_array = [x,y,x+size_box,y,x+size_box,y+size_box,x,y+size_box]
                category_id = i #номер категории[0-20]
                id_segmentation = rgb_list_21[i][0] + rgb_list_21[i][1]*256 + rgb_list_21[i][2]*256*256 #мапинг с PNG через RGB: ids=R+G*256+B*256^2
                segmentations_list.append({'id': id_segmentation,
                                           'category_id': category_id,
                                           'area':0,
                                           'bbox':[x,y,size_box,size_box],
                                           'bbox_mode': "<BoxMode.XYWH_ABS: 1>",
                                           'iscrowd':0,
                                           'segmentation':segmentation_array})
                #update array2png
                array2png[y:y+size_box, x:x+size_box] = rgb_list_21[i]
        
        #create PNG
        img = Image.fromarray(array2png, 'RGB')
        path2origFile = face_ann['img_path']
        path2origFile = path2origFile.split("/")
        new_path_file_name = dirForNewPNGs + path2origFile[2].split(".")[0] + '.png'
        new_file_name = path2origFile[2].split(".")[0] + '.png'
        img.save(new_path_file_name)
        
        annotations_list.append({'image_id': image_id,
                                 'file_name': new_file_name, #PNG file name,
                                 'segments_info': segmentations_list})
                                 #'num_keypoints': len(face_ann['keypoints']),
                                 #'area': 0,
                                 #'iscrowd': 0,
                                 #'keypoints': face_ann['keypoints'],
                                 #'bbox': face_ann['bbox'],
                                 #'category_id': 0})

    # Build COCO-like dictionary
    dataset_dict = dict()

    # =============================== Dataset Info =============================== #
    dataset_info = {
        'description': 'Annotated Facial Landmarks in the Wild (AFLW)',
        'url': 'https://www.tugraz.at/institute/icg/research/team-bischof/lrs/downloads/aflw/',
        'version': '1.0',
        'year': 2011,
        'contributor': '',
        'date_created': '2011'
    }
    dataset_dict.update(dataset_info)

    # ============================= Dataset Licenses ============================= #
    dataset_licenses = {
        'licenses': [
            {'id': 0,
             'url': 'https://www.tugraz.at/institute/icg/research/team-bischof/lrs/downloads/aflw/',
             'name': 'aflw_license'}
        ]
    }
    dataset_dict.update(dataset_licenses)

    # ============================== Dataset Images ============================== #
    dataset_images = {'images': images_list}
    dataset_dict.update(dataset_images)

    # =========================== Dataset Annotations ============================ #
    dataset_annotations = {'annotations': annotations_list}
    dataset_dict.update(dataset_annotations)

    # ============================ Dataset Categories ============================ #
    all_keypoints = ['LeftBrowLeftCorner',
                     'LeftBrowCenter',
                     'LeftBrowRightCorner',
                     'RightBrowLeftCorner',
                     'RightBrowCenter',
                     'RightBrowRightCorner',
                     'LeftEyeLeftCorner',
                     'LeftEyeCenter',
                     'LeftEyeRightCorner',
                     'RightEyeLeftCorner',
                     'RightEyeCenter',
                     'RightEyeRightCorner',
                     'LeftEar',
                     'NoseLeft',
                     'NoseCenter',
                     'NoseRight',
                     'RightEar',
                     'MouthLeftCorner',
                     'MouthCenter',
                     'MouthRightCorner',
                     'ChinCenter']
    list_categories = []
    for i, category in enumerate(all_keypoints):
        list_categories.append({'id': i,
                                'name': category,
                                'supercategory': category,
                                'isthing': 0, #если не отображается сменить на [0,1]
                                'color': rgb_list_21[i]})
    dataset_categories = {'categories': list_categories}
    dataset_dict.update(dataset_categories)

    
    if isVerbose:
        print("Done!")

    # Save dataset dictionary as json file
    if isVerbose:
        print("  \\__Save dataset dictionary as json file...", end="")
        sys.stdout.flush()

    with open(jsonFile, 'w') as fp:
        json.dump(dataset_dict, fp)

    if isVerbose:
        print("Done!")

In [39]:
main()

#. Transform AFLW annotations into COCO json format...
  \__Open the AFLW SQLight database...Done!
  \__Count total number of images in AFLW database: 24384
  \__Populate AFLW dataset dictionary...[####################] 100% 
  \__Update AFLW dataset dictionary with keypoints...Done!
  \__Close the AFLW SQLight database...Done!
  \__Convert to COCO format...Done!
  \__Save dataset dictionary as json file...Done!


In [40]:
size_box = 6

def main_instance():
    # Set up a parser for command line arguments
    isVerbose = 1
    dataset_root = "/Users/nikitakorobkov/Downloads/aflw/data"
    jsonFile = "panoptic_aflwcoco8.json"
    dirForNewJPGs = "allImagesJPG3/"
    dirForNewPNGs = "allImagesPNG3/"

    if isVerbose:
        print("#. Transform AFLW annotations into COCO json format...")

    # Open the original AFLW annotation (sqlite database)
    if isVerbose:
        print("  \\__Open the AFLW SQLight database...", end="")
        sys.stdout.flush()

    conn = sqlite3.connect(osp.join(dataset_root, 'aflw.sqlite'))
    cursor = conn.cursor()
    if isVerbose:
        print("Done!")

    # Build sqlite queries
    select_str = "faces.face_id, " \
                 "imgs.filepath, " \
                 "rect.x, rect.y, " \
                 "rect.w, " \
                 "rect.h, " \
                 "pose.roll, " \
                 "pose.pitch, " \
                 "pose.yaw, " \
                 "metadata.sex"
    from_str = "faces, " \
               "faceimages " \
               "imgs, " \
               "facerect rect, " \
               "facepose pose, " \
               "facemetadata metadata"
    where_str = "faces.file_id = imgs.file_id and " \
                "faces.face_id = rect.face_id and " \
                "faces.face_id = pose.face_id and " \
                "faces.face_id = metadata.face_id"
    query_res = exec_sqlite_query(cursor, select_str, from_str, where_str)

    # Count total number of images in AFLW dataset
    if isVerbose:
        print("  \\__Count total number of images in AFLW database: ", end="")
        sys.stdout.flush()
    total_num_images = 0
    for _ in query_res:
        total_num_images += 1
    if isVerbose:
        print(total_num_images)

    # Output file for appending the file paths of not found images
    not_found_images_file = 'not_found_images_aflw.txt'
    try:
        os.remove(not_found_images_file)
    except OSError:
        pass

    # Temporary dataset variables
    aflw_dataset_dict = dict()

    # Register to dataset_dict
    img_cnt = 0
    for face_id, path, rectx, recty, rectw, recth, roll, pitch, yaw, gender in query_res:

        img_cnt += 1

        # Get current image path
        img_path = osp.join(dataset_root, 'flickr', path)

        # Process current image
        if osp.isfile(img_path):
            img_w, img_h = get_img_size(img_path)

            keypoints = N_LANDMARK * 3 * [0]
            pose = [roll, pitch, yaw]
            gender = 0 if gender == 'm' else 1

            # Register
            aflw_dataset_dict[face_id] = {
                'face_id': face_id,
                'img_path': osp.join('flickr', path),
                'width': img_w,
                'height': img_h,
                'bbox': (rectx, recty, rectw, recth),
                'keypoints': keypoints,
                'pose': pose,
                'gender': gender}

        # If current image file does not exist append the not found images filepaths to `not_found_images_file` and
        # continue with the next image file.
        else:
            with open(not_found_images_file, "a") as out:
                out.write("%s\n" % img_path)
            continue

        # Show progress bar
        if isVerbose:
            progress_updt("  \\__Populate AFLW dataset dictionary...", total_num_images, img_cnt)

    if isVerbose:
        print("  \\__Update AFLW dataset dictionary with keypoints...", end="")
        sys.stdout.flush()

    # Landmark property
    # (Visibility is expressed by lack of the coordinate's row.)
    select_str = "faces.face_id, coords.feature_id, coords.x, coords.y"
    from_str = "faces, featurecoords coords"
    where_str = "faces.face_id = coords.face_id"
    query_res = exec_sqlite_query(cursor, select_str, from_str, where_str)

    # Register to dataset_dict
    invalid_face_ids = list()
    for face_id, feature_id, x, y in query_res:
        assert (1 <= feature_id <= N_LANDMARK)
        if face_id in aflw_dataset_dict:
            idx = feature_id - 1
            aflw_dataset_dict[face_id]['keypoints'][3 * idx] = x
            aflw_dataset_dict[face_id]['keypoints'][3 * idx + 1] = y
            aflw_dataset_dict[face_id]['keypoints'][3 * idx + 2] = 1

        elif face_id not in invalid_face_ids:
            invalid_face_ids.append(face_id)

    if isVerbose:
        print("Done!")

    # Close database
    if isVerbose:
        print("  \\__Close the AFLW SQLight database...", end="")
        sys.stdout.flush()
    cursor.close()
    if isVerbose:
        print("Done!")

    # Close database
    if isVerbose:
        print("  \\__Convert to COCO format...", end="")
        sys.stdout.flush()

    images_list = []
    annotations_list = []
    iterI = 0
    for face_id, face_ann in aflw_dataset_dict.items():
        if iterI > 9998:
            break
        iterI = iterI + 1
        
        img_dir_num = int(face_ann['img_path'].split("/")[1])
        img_file_num = int(re.findall(r'\d+', face_ann['img_path'].split("/")[-1].split(".")[0])[0])
        image_id = int("%d%05d" % (img_dir_num, img_file_num))

        #save jpg
        new_file_name = face_ann['img_path'].split("/")[2]
        if(new_file_name.split(".")[1] == "png"):
            new_file_name = new_file_name.split(".")[0] + ".jpg"
            
        new_path_file = "allImagesJPG3/" + new_file_name
        copyfile(face_ann['img_path'], new_path_file)
        
        images_list.append({'id': image_id,
                            'file_name': new_file_name,
                            'height': face_ann['height'],
                            'width': face_ann['width'],
                            'date_captured': '',
                            'flickr_url': '',
                            'license': 1,
                            'dataset': 'aflw'})

        list_keypoints = []
        one_keypoint = []
        for value in face_ann['keypoints']:
            one_keypoint.append(value)
            if len(one_keypoint)==3:
                list_keypoints.append(one_keypoint)
                one_keypoint= []

        #create Array2PNG
        w, h = face_ann['width'], face_ann['height']
        array2png = np.zeros((h, w, 3), dtype=np.uint8)
        
        segmentations_list = []
        for i, keypoint in enumerate(list_keypoints):
            v=keypoint[2]
            if v == 1: #видно keypoint
                x=int(keypoint[0])
                y=int(keypoint[1])
                segmentation_array = [x,y,x+size_box,y,x+size_box,y+size_box,x,y+size_box]
                category_id = i #номер категории[0-20]
                id_segmentation = rgb_list_21[i][0] + rgb_list_21[i][1]*256 + rgb_list_21[i][2]*256*256 #мапинг с PNG через RGB: ids=R+G*256+B*256^2
                segmentations_list.append({'id': id_segmentation,
                                           'category_id': category_id,
                                           'area':0,
                                           'bbox':[x,y,size_box,size_box],
                                           'bbox_mode': "<BoxMode.XYWH_ABS: 1>",
                                           'iscrowd':0,
                                           'segmentation':segmentation_array})
                #update array2png
                array2png[y:y+size_box, x:x+size_box] = rgb_list_21[i]
        
        #create PNG
        img = Image.fromarray(array2png, 'RGB')
        path2origFile = face_ann['img_path']
        path2origFile = path2origFile.split("/")
        new_path_file_name = dirForNewPNGs + path2origFile[2].split(".")[0] + '.png'
        new_file_name = path2origFile[2].split(".")[0] + '.png'
        img.save(new_path_file_name)
        
        annotations_list.append({'id': face_id,
                                 'image_id': image_id,
                                 'file_name': new_file_name, #PNG file name,
                                 'segments_info': segmentations_list})
                                 #'num_keypoints': len(face_ann['keypoints']),
                                 #'area': 0,
                                 #'iscrowd': 0,
                                 #'keypoints': face_ann['keypoints'],
                                 #'bbox': face_ann['bbox'],
                                 #'category_id': 0})

    # Build COCO-like dictionary
    dataset_dict = dict()

    # =============================== Dataset Info =============================== #
    dataset_info = {
        'description': 'Annotated Facial Landmarks in the Wild (AFLW)',
        'url': 'https://www.tugraz.at/institute/icg/research/team-bischof/lrs/downloads/aflw/',
        'version': '1.0',
        'year': 2011,
        'contributor': '',
        'date_created': '2011'
    }
    dataset_dict.update(dataset_info)

    # ============================= Dataset Licenses ============================= #
    dataset_licenses = {
        'licenses': [
            {'id': 0,
             'url': 'https://www.tugraz.at/institute/icg/research/team-bischof/lrs/downloads/aflw/',
             'name': 'aflw_license'}
        ]
    }
    dataset_dict.update(dataset_licenses)

    # ============================== Dataset Images ============================== #
    dataset_images = {'images': images_list}
    dataset_dict.update(dataset_images)

    # =========================== Dataset Annotations ============================ #
    dataset_annotations = {'annotations': annotations_list}
    dataset_dict.update(dataset_annotations)

    # ============================ Dataset Categories ============================ #
    all_keypoints = ['LeftBrowLeftCorner',
                     'LeftBrowCenter',
                     'LeftBrowRightCorner',
                     'RightBrowLeftCorner',
                     'RightBrowCenter',
                     'RightBrowRightCorner',
                     'LeftEyeLeftCorner',
                     'LeftEyeCenter',
                     'LeftEyeRightCorner',
                     'RightEyeLeftCorner',
                     'RightEyeCenter',
                     'RightEyeRightCorner',
                     'LeftEar',
                     'NoseLeft',
                     'NoseCenter',
                     'NoseRight',
                     'RightEar',
                     'MouthLeftCorner',
                     'MouthCenter',
                     'MouthRightCorner',
                     'ChinCenter']
    list_categories = []
    for i, category in enumerate(all_keypoints):
        list_categories.append({'id': i,
                                'name': category,
                                'supercategory': category,
                                'isthing': 0, #если не отображается сменить на [0,1]
                                'color': rgb_list_21[i]})
    dataset_categories = {'categories': list_categories}
    dataset_dict.update(dataset_categories)

    
    if isVerbose:
        print("Done!")

    # Save dataset dictionary as json file
    if isVerbose:
        print("  \\__Save dataset dictionary as json file...", end="")
        sys.stdout.flush()

    with open(jsonFile, 'w') as fp:
        json.dump(dataset_dict, fp)

    if isVerbose:
        print("Done!")

In [42]:
main_instance()

#. Transform AFLW annotations into COCO json format...
  \__Open the AFLW SQLight database...Done!
  \__Count total number of images in AFLW database: 24384
  \__Populate AFLW dataset dictionary...[####################] 100% 
  \__Update AFLW dataset dictionary with keypoints...Done!
  \__Close the AFLW SQLight database...Done!
  \__Convert to COCO format...Done!
  \__Save dataset dictionary as json file...Done!


In [73]:
def main2():
    # Set up a parser for command line arguments
    isVerbose = 1
    dataset_root = "/Users/nikitakorobkov/Downloads/aflw/data"
    json_file = "aflw_coco12.json"

    if isVerbose:
        print("#. Transform AFLW annotations into COCO json format...")

    # Open the original AFLW annotation (sqlite database)
    if isVerbose:
        print("  \\__Open the AFLW SQLight database...", end="")
        sys.stdout.flush()

    conn = sqlite3.connect(osp.join(dataset_root, 'aflw.sqlite'))
    cursor = conn.cursor()
    if isVerbose:
        print("Done!")

    # Build sqlite queries
    select_str = "faces.face_id, " \
                 "imgs.filepath, " \
                 "rect.x, rect.y, " \
                 "rect.w, " \
                 "rect.h, " \
                 "pose.roll, " \
                 "pose.pitch, " \
                 "pose.yaw, " \
                 "metadata.sex"
    from_str = "faces, " \
               "faceimages " \
               "imgs, " \
               "facerect rect, " \
               "facepose pose, " \
               "facemetadata metadata"
    where_str = "faces.file_id = imgs.file_id and " \
                "faces.face_id = rect.face_id and " \
                "faces.face_id = pose.face_id and " \
                "faces.face_id = metadata.face_id"
    query_res = exec_sqlite_query(cursor, select_str, from_str, where_str)

    # Count total number of images in AFLW dataset
    if isVerbose:
        print("  \\__Count total number of images in AFLW database: ", end="")
        sys.stdout.flush()
    total_num_images = 0
    for _ in query_res:
        total_num_images += 1
    if isVerbose:
        print(total_num_images)

    # Output file for appending the file paths of not found images
    not_found_images_file = 'not_found_images_aflw.txt'
    try:
        os.remove(not_found_images_file)
    except OSError:
        pass

    # Temporary dataset variables
    aflw_dataset_dict = dict()

    # Register to dataset_dict
    img_cnt = 0
    for face_id, path, rectx, recty, rectw, recth, roll, pitch, yaw, gender in query_res:

        img_cnt += 1

        # Get current image path
        img_path = osp.join(dataset_root, 'flickr', path)

        # Process current image
        if osp.isfile(img_path):
            img_w, img_h = get_img_size(img_path)

            keypoints = N_LANDMARK * 3 * [0]
            pose = [roll, pitch, yaw]
            gender = 0 if gender == 'm' else 1

            # Register
            aflw_dataset_dict[face_id] = {
                'face_id': face_id,
                'img_path': osp.join('flickr', path),
                'width': img_w,
                'height': img_h,
                'bbox': (rectx, recty, rectw, recth),
                'keypoints': keypoints,
                'pose': pose,
                'gender': gender}

        # If current image file does not exist append the not found images filepaths to `not_found_images_file` and
        # continue with the next image file.
        else:
            with open(not_found_images_file, "a") as out:
                out.write("%s\n" % img_path)
            continue

        # Show progress bar
        if isVerbose:
            progress_updt("  \\__Populate AFLW dataset dictionary...", total_num_images, img_cnt)

    if isVerbose:
        print("  \\__Update AFLW dataset dictionary with keypoints...", end="")
        sys.stdout.flush()

    # Landmark property
    # (Visibility is expressed by lack of the coordinate's row.)
    select_str = "faces.face_id, coords.feature_id, coords.x, coords.y"
    from_str = "faces, featurecoords coords"
    where_str = "faces.face_id = coords.face_id"
    query_res = exec_sqlite_query(cursor, select_str, from_str, where_str)

    # Register to dataset_dict
    invalid_face_ids = list()
    for face_id, feature_id, x, y in query_res:
        assert (1 <= feature_id <= N_LANDMARK)
        if face_id in aflw_dataset_dict:
            idx = feature_id - 1
            aflw_dataset_dict[face_id]['keypoints'][3 * idx] = x
            aflw_dataset_dict[face_id]['keypoints'][3 * idx + 1] = y
            aflw_dataset_dict[face_id]['keypoints'][3 * idx + 2] = 1

        elif face_id not in invalid_face_ids:
            invalid_face_ids.append(face_id)

    if isVerbose:
        print("Done!")

    # Close database
    if isVerbose:
        print("  \\__Close the AFLW SQLight database...", end="")
        sys.stdout.flush()
    cursor.close()
    if isVerbose:
        print("Done!")

    # Close database
    if isVerbose:
        print("  \\__Convert to COCO format...", end="")
        sys.stdout.flush()

    images_list = []
    annotations_list = []
    iterI = 0
    id_iter = 0
    for face_id, face_ann in aflw_dataset_dict.items():
        if iterI > 9998:
            break
        iterI = iterI + 1
        
        img_dir_num = int(face_ann['img_path'].split("/")[1])
        img_file_num = int(re.findall(r'\d+', face_ann['img_path'].split("/")[-1].split(".")[0])[0])
        image_id = int("%d%05d" % (img_dir_num, img_file_num))

        #save jpg
        new_file_name = face_ann['img_path'].split("/")[2]
        if(new_file_name.split(".")[1] == "png"):
            new_file_name = new_file_name.split(".")[0] + ".jpg"
            
        new_path_file = "allImagesJPG3/" + new_file_name
        #copyfile(face_ann['img_path'], new_path_file)
        
        list_keypoints = []
        one_keypoint = []
        for value in face_ann['keypoints']:
            one_keypoint.append(value)
            if len(one_keypoint)==3:
                list_keypoints.append(one_keypoint)
                one_keypoint= []

        images_list.append({'id': image_id,
                            'file_name': new_file_name,
                            'height': face_ann['height'],
                            'width': face_ann['width'],
                            'date_captured': '',
                            'flickr_url': '',
                            'license': 1,
                            'dataset': 'aflw'})
        
        segmentations_list = []
        for i, keypoint in enumerate(list_keypoints):
            v=keypoint[2]
            if v == 1: #видно keypoint
                x=int(keypoint[0])
                y=int(keypoint[1])
                segmentation_array = [[x,y,x+size_box,y,x+size_box,y+size_box,x,y+size_box]]
                category_id = i #номер категории[0-20]
            
                id_segmentation = rgb_list_21[i][0] + rgb_list_21[i][1]*256 + rgb_list_21[i][2]*256*256 #мапинг с PNG через RGB: ids=R+G*256+B*256^2
                annotations_list.append({'id': id_iter+500000,
                                         'image_id': image_id,
                                         'category_id': category_id,
                                         'area':0,
                                         'bbox':[x,y,size_box,size_box],
                                         'iscrowd':0,
                                         'segmentation':segmentation_array})
                id_iter = id_iter+1
        
        x1 = face_ann['bbox'][0]
        y1 = face_ann['bbox'][1]
        x2 = face_ann['bbox'][0] + face_ann['bbox'][2]
        y2 = face_ann['bbox'][1] + face_ann['bbox'][3]
        
        annotations_list.append({'id': face_id,
                                 'image_id': image_id,
                                 'segmentation': [[x1,y1,x2,y1,x2,y2,x1,y2]],
                                 #'num_keypoints': len(face_ann['keypoints']),
                                 'area': 0,
                                 'iscrowd': 0,
                                 #'keypoints': face_ann['keypoints'],
                                 'bbox': face_ann['bbox'],
                                 'category_id': 21}) #21 - face

    # Build COCO-like dictionary
    dataset_dict = dict()

    # =============================== Dataset Info =============================== #
    dataset_info = {
        'description': 'Annotated Facial Landmarks in the Wild (AFLW)',
        'url': 'https://www.tugraz.at/institute/icg/research/team-bischof/lrs/downloads/aflw/',
        'version': '1.0',
        'year': 2011,
        'contributor': '',
        'date_created': '2011'
    }
    dataset_dict.update(dataset_info)

    # ============================= Dataset Licenses ============================= #
    dataset_licenses = {
        'licenses': [
            {'id': 0,
             'url': 'https://www.tugraz.at/institute/icg/research/team-bischof/lrs/downloads/aflw/',
             'name': 'aflw_license'}
        ]
    }
    dataset_dict.update(dataset_licenses)

    # ============================== Dataset Images ============================== #
    dataset_images = {'images': images_list}
    dataset_dict.update(dataset_images)

    # =========================== Dataset Annotations ============================ #
    dataset_annotations = {'annotations': annotations_list}
    dataset_dict.update(dataset_annotations)

    # ============================ Dataset Categories ============================ #
    all_keypoints = ['LeftBrowLeftCorner',
                     'LeftBrowCenter',
                     'LeftBrowRightCorner',
                     'RightBrowLeftCorner',
                     'RightBrowCenter',
                     'RightBrowRightCorner',
                     'LeftEyeLeftCorner',
                     'LeftEyeCenter',
                     'LeftEyeRightCorner',
                     'RightEyeLeftCorner',
                     'RightEyeCenter',
                     'RightEyeRightCorner',
                     'LeftEar',
                     'NoseLeft',
                     'NoseCenter',
                     'NoseRight',
                     'RightEar',
                     'MouthLeftCorner',
                     'MouthCenter',
                     'MouthRightCorner',
                     'ChinCenter',
                     'Face']
    list_categories = []
    for i, category in enumerate(all_keypoints):
        list_categories.append({'id': i,
                                'name': category,
                                'supercategory': category,
                                'isthing': 0, #если не отображается сменить на [0,1]
                                'color': rgb_list_21[i]})
    dataset_categories = {'categories': list_categories}
    dataset_dict.update(dataset_categories)

    if isVerbose:
        print("Done!")

    # Save dataset dictionary as json file
    if isVerbose:
        print("  \\__Save dataset dictionary as json file...", end="")
        sys.stdout.flush()

    with open(json_file, 'w') as fp:
        json.dump(dataset_dict, fp)

    if isVerbose:
        print("Done!")

In [74]:
main2()

#. Transform AFLW annotations into COCO json format...
  \__Open the AFLW SQLight database...Done!
  \__Count total number of images in AFLW database: 24384
  \__Populate AFLW dataset dictionary...[###############-----] 75% 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  \__Populate AFLW dataset dictionary...[##################--] 91% 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  \__Populate AFLW dataset dictionary...[####################] 100% 
  \__Update AFLW dataset dictionary with keypoints...Done!
  \__Close the AFLW SQLight database...Done!
  \__Convert to COCO format...Done!
  \__Save dataset dictionary as json file...Done!


In [61]:
size_box = 6

def main_instPanoptic():
    # Set up a parser for command line arguments
    isVerbose = 1
    dataset_root = "/Users/nikitakorobkov/Downloads/aflw/data"
    jsonFile = "panoptic_aflw4coco.json"
    dirForNewPNGs = "allImagesPNG5/"

    if isVerbose:
        print("#. Transform AFLW annotations into COCO json format...")

    # Open the original AFLW annotation (sqlite database)
    if isVerbose:
        print("  \\__Open the AFLW SQLight database...", end="")
        sys.stdout.flush()

    conn = sqlite3.connect(osp.join(dataset_root, 'aflw.sqlite'))
    cursor = conn.cursor()
    if isVerbose:
        print("Done!")

    # Build sqlite queries
    select_str = "faces.face_id, " \
                 "imgs.filepath, " \
                 "rect.x, rect.y, " \
                 "rect.w, " \
                 "rect.h, " \
                 "pose.roll, " \
                 "pose.pitch, " \
                 "pose.yaw, " \
                 "metadata.sex"
    from_str = "faces, " \
               "faceimages " \
               "imgs, " \
               "facerect rect, " \
               "facepose pose, " \
               "facemetadata metadata"
    where_str = "faces.file_id = imgs.file_id and " \
                "faces.face_id = rect.face_id and " \
                "faces.face_id = pose.face_id and " \
                "faces.face_id = metadata.face_id"
    query_res = exec_sqlite_query(cursor, select_str, from_str, where_str)

    # Count total number of images in AFLW dataset
    if isVerbose:
        print("  \\__Count total number of images in AFLW database: ", end="")
        sys.stdout.flush()
    total_num_images = 0
    for _ in query_res:
        total_num_images += 1
    if isVerbose:
        print(total_num_images)

    # Output file for appending the file paths of not found images
    not_found_images_file = 'not_found_images_aflw.txt'
    try:
        os.remove(not_found_images_file)
    except OSError:
        pass

    # Temporary dataset variables
    aflw_dataset_dict = dict()

    # Register to dataset_dict
    img_cnt = 0
    for face_id, path, rectx, recty, rectw, recth, roll, pitch, yaw, gender in query_res:

        img_cnt += 1

        # Get current image path
        img_path = osp.join(dataset_root, 'flickr', path)

        # Process current image
        if osp.isfile(img_path):
            img_w, img_h = get_img_size(img_path)

            keypoints = N_LANDMARK * 3 * [0]
            pose = [roll, pitch, yaw]
            gender = 0 if gender == 'm' else 1

            # Register
            aflw_dataset_dict[face_id] = {
                'face_id': face_id,
                'img_path': osp.join('flickr', path),
                'width': img_w,
                'height': img_h,
                'bbox': (rectx, recty, rectw, recth),
                'keypoints': keypoints,
                'pose': pose,
                'gender': gender}

        # If current image file does not exist append the not found images filepaths to `not_found_images_file` and
        # continue with the next image file.
        else:
            with open(not_found_images_file, "a") as out:
                out.write("%s\n" % img_path)
            continue

        # Show progress bar
        if isVerbose:
            progress_updt("  \\__Populate AFLW dataset dictionary...", total_num_images, img_cnt)

    if isVerbose:
        print("  \\__Update AFLW dataset dictionary with keypoints...", end="")
        sys.stdout.flush()

    # Landmark property
    # (Visibility is expressed by lack of the coordinate's row.)
    select_str = "faces.face_id, coords.feature_id, coords.x, coords.y"
    from_str = "faces, featurecoords coords"
    where_str = "faces.face_id = coords.face_id"
    query_res = exec_sqlite_query(cursor, select_str, from_str, where_str)

    # Register to dataset_dict
    invalid_face_ids = list()
    for face_id, feature_id, x, y in query_res:
        assert (1 <= feature_id <= N_LANDMARK)
        if face_id in aflw_dataset_dict:
            idx = feature_id - 1
            aflw_dataset_dict[face_id]['keypoints'][3 * idx] = x
            aflw_dataset_dict[face_id]['keypoints'][3 * idx + 1] = y
            aflw_dataset_dict[face_id]['keypoints'][3 * idx + 2] = 1

        elif face_id not in invalid_face_ids:
            invalid_face_ids.append(face_id)

    if isVerbose:
        print("Done!")

    # Close database
    if isVerbose:
        print("  \\__Close the AFLW SQLight database...", end="")
        sys.stdout.flush()
    cursor.close()
    if isVerbose:
        print("Done!")

    # Close database
    if isVerbose:
        print("  \\__Convert to COCO format...", end="")
        sys.stdout.flush()

    images_list = []
    annotations_list = []
    iterI = 0
    for face_id, face_ann in aflw_dataset_dict.items():
        if iterI > 9998:
            break
        iterI = iterI + 1
        
        img_dir_num = int(face_ann['img_path'].split("/")[1])
        img_file_num = int(re.findall(r'\d+', face_ann['img_path'].split("/")[-1].split(".")[0])[0])
        image_id = int("%d%05d" % (img_dir_num, img_file_num))

        #save jpg
        new_path_file = dirForNewPNGs + face_ann['img_path'].split("/")[2]

        images_list.append({'id': image_id,
                            'file_name': new_path_file,
                            'height': face_ann['height'],
                            'width': face_ann['width'],
                            'date_captured': '',
                            'flickr_url': '',
                            'license': 1,
                            'dataset': 'aflw'})

        list_keypoints = []
        one_keypoint = []
        for value in face_ann['keypoints']:
            one_keypoint.append(value)
            if len(one_keypoint)==3:
                list_keypoints.append(one_keypoint)
                one_keypoint= []

        #create Array2PNG
        w, h = face_ann['width'], face_ann['height']
        array2png = np.zeros((h, w, 3), dtype=np.uint8)
        
        segmentations_list = []
        for i, keypoint in enumerate(list_keypoints):
            v=keypoint[2]
            if v == 1: #видно keypoint
                x=int(keypoint[0])
                y=int(keypoint[1])
                segmentation_array = [[x,y,x+size_box,y,x+size_box,y+size_box,x,y+size_box]]
                category_id = i #номер категории[0-20]
                id_segmentation = rgb_list_21[i][0] + rgb_list_21[i][1]*256 + rgb_list_21[i][2]*256*256 #мапинг с PNG через RGB: ids=R+G*256+B*256^2
                segmentations_list.append({'id': id_segmentation,
                                           'category_id': category_id,
                                           'area':0,
                                           'bbox':[x,y,size_box,size_box],
                                           'bbox_mode': "<BoxMode.XYWH_ABS: 1>",
                                           'iscrowd':0,
                                           'segmentation':segmentation_array})
                #update array2png
                array2png[y:y+size_box, x:x+size_box] = rgb_list_21[i]
        
        #create PNG
        img = Image.fromarray(array2png, 'RGB')
        path2origFile = face_ann['img_path']
        path2origFile = path2origFile.split("/")
        new_path_file_name = dirForPathPNGs + path2origFile[2].split(".")[0] + '.png'
        new_file_name = path2origFile[2].split(".")[0] + '.png'
        img.save(new_path_file_name)
        
        annotations_list.append({'image_id': image_id,
                                 'file_name': new_file_name, #PNG file name,
                                 'segments_info': segmentations_list})
                                 #'num_keypoints': len(face_ann['keypoints']),
                                 #'area': 0,
                                 #'iscrowd': 0,
                                 #'keypoints': face_ann['keypoints'],
                                 #'bbox': face_ann['bbox'],
                                 #'category_id': 0})

    # Build COCO-like dictionary
    dataset_dict = dict()

    # =============================== Dataset Info =============================== #
    dataset_info = {
        'description': 'Annotated Facial Landmarks in the Wild (AFLW)',
        'url': 'https://www.tugraz.at/institute/icg/research/team-bischof/lrs/downloads/aflw/',
        'version': '1.0',
        'year': 2011,
        'contributor': '',
        'date_created': '2011'
    }
    dataset_dict.update(dataset_info)

    # ============================= Dataset Licenses ============================= #
    dataset_licenses = {
        'licenses': [
            {'id': 0,
             'url': 'https://www.tugraz.at/institute/icg/research/team-bischof/lrs/downloads/aflw/',
             'name': 'aflw_license'}
        ]
    }
    dataset_dict.update(dataset_licenses)

    # ============================== Dataset Images ============================== #
    dataset_images = {'images': images_list}
    dataset_dict.update(dataset_images)

    # =========================== Dataset Annotations ============================ #
    dataset_annotations = {'annotation': annotations_list}
    dataset_dict.update(dataset_annotations)

    # ============================ Dataset Categories ============================ #
    all_keypoints = ['LeftBrowLeftCorner',
                     'LeftBrowCenter',
                     'LeftBrowRightCorner',
                     'RightBrowLeftCorner',
                     'RightBrowCenter',
                     'RightBrowRightCorner',
                     'LeftEyeLeftCorner',
                     'LeftEyeCenter',
                     'LeftEyeRightCorner',
                     'RightEyeLeftCorner',
                     'RightEyeCenter',
                     'RightEyeRightCorner',
                     'LeftEar',
                     'NoseLeft',
                     'NoseCenter',
                     'NoseRight',
                     'RightEar',
                     'MouthLeftCorner',
                     'MouthCenter',
                     'MouthRightCorner',
                     'ChinCenter']
    list_categories = []
    for i, category in enumerate(all_keypoints):
        list_categories.append({'id': i,
                                'name': category,
                                'supercategory': category,
                                'isthing': 0, #если не отображается сменить на [0,1]
                                'color': rgb_list_21[i]})
    dataset_categories = {'categories': list_categories}
    dataset_dict.update(dataset_categories)

    
    if isVerbose:
        print("Done!")

    # Save dataset dictionary as json file
    if isVerbose:
        print("  \\__Save dataset dictionary as json file...", end="")
        sys.stdout.flush()

    with open(jsonFile, 'w') as fp:
        json.dump(dataset_dict, fp)

    if isVerbose:
        print("Done!")